<a href="https://colab.research.google.com/github/WhaiPkuKeyuhao/Unsloth_Tut/blob/main/nb/GRPO_5_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [3]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

Load up `Llama 3.1 8B Instruct`, and set parameters

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 16 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen3-1.7B",
    max_seq_length = max_seq_length,
    dtype = torch.float16,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = False, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.78, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-13 01:27:15 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-13 01:27:15 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.5.1: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.41G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

Unsloth 2025.5.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
def extract_final_answer(string):
    try:
        string_list = string.split("</think>")
        if len(string_list) == 2:
            return string_list[1]
        else:
            return "No Final Answer"
    except Exception as e:
        print(e)
        return "No Final Answer"

def extract_reasoning(string):
    try:
        string_list = string.split("</think>")
        if len(string_list) == 2:
            return string_list[0]
        else:
            return "No Reasoning"
    except Exception as e:
        print(e)
        return "No Reasoning"

In [11]:
from datasets import Dataset, load_dataset
dataset = load_dataset("json", data_files="/content/drive/MyDrive/llm_distill/data_collected/accumulative_data/training_set.json")

Thinking_start = "<think>"
Thinking_end   = "</think>"

system_prompt = \
f"""You are given a problem.
Think about the problem and then provide your final answer.
Place the content of thinking between {Thinking_start} and {Thinking_end}.
Then, provide the final answer directly.
"""

dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["input"]},
    ],
    "answer": extract_final_answer(x["output"]),
    "reasoning":extract_reasoning(x["output"])
})


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.

Let's look at the first row:

In [29]:
dataset[0]

{'input': '我正在模拟一个智能体的一天。这个智能体名字叫唐唯燕，女性，67岁。最高学历为高中。婚姻状态为已婚。当前的人生阶段为自由职业。在星期六凌晨0点到上午九点这9个小时中，她睡觉0.25个小时。在早上6点到9点这3个小时中，她吃饭2.75个小时。在上午11点到下午1点这两个小时中，她吃饭0.25小时。在早上9点到下午6点这9小时中，她娱乐2.75个小时，工作0.5小时，学习0.25小时，。请思考每个提到的动作持续时间的合理性并给出一个分数。0分为非常不正常，100分为非常正常。最后将这些单项的分数求均值得到一个综合起来的分数。在最终的回复中只给出这个综合的分数，不给出任何的理由。',
 'output': '<think>好的，我现在需要分析用户提供的唐唯燕的一天活动时间安排，并给每个动作的持续时间合理性打分，最后计算综合分数。首先，用户给出的时间段和活动如下：\n\n1. **星期六凌晨0点到上午9点（9小时）**：睡觉0.25小时（即15分钟）。这显然有问题，因为正常成年人每天需要约7-9小时睡眠，而这里只有15分钟，明显不合理，可能得0分。\n\n2. **早上6点到9点（3小时）**：吃饭用了2.75小时，即2小时45分钟。通常早餐时间在30分钟到1小时左右，这么长时间吃饭不太寻常，可能得分较低，比如20分。\n\n3. **上午11点到下午1点（2小时）**：吃饭0.25小时（15分钟）。午餐时间15分钟比较合理，可能得80分。\n\n4. **早上9点到下午6点（9小时）**：\n   - 娱乐2.75小时：约2小时45分钟，属于正常休闲时间，可能得90分。\n   - 工作0.5小时（30分钟）：作为自由职业者，工作时间太短，可能得30分。\n   - 学习0.25小时（15分钟）：每天学习15分钟可能较少，但合理，可能得60分。\n\n接下来，计算各单项分数：\n- 睡觉：0分\n- 早餐：20分\n- 午餐：80分\n- 娱乐：90分\n- 工作：30分\n- 学习：60分\n\n综合分数：(0 + 20 + 80 + 90 + 30 + 60) / 6 = 280 / 6 ≈ 46.67，四舍五入约47分。</think>46',
 'prompt': [{'content': 'You are given a problem.\n

Let's map the dataset! and see the first row:

We create a regex format to match the reasoning sections and answers:

In [30]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{Thinking_start}.+?{Thinking_end}.*?"\
    rf"(.+?)"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

We verify it works:

In [31]:
match_format.search(
    "<think>Let me think!</think>"\
    "2",
)

<re.Match object; span=(0, 29), match='<think>Let me think!</think>2'>

We now want to create a reward function to match the format exactly - we reward it with 3 points if it succeeds:

In [32]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

Finally, we want to extract the generated answer, and reward or penalize it! We also reward it based on how close the answer is to the true one via ratios:

In [33]:
def check_answer(prompts, completions, answer, **kwargs):
    scores = []
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]
    generated_answers = [extract_final_answer(response).strip() for response in responses]

    for generated_answer, labeled_answer in zip(generated_answers,answer):
        try:
            diff = abs(int(generated_answer) - int(labeled_answer))
            if diff < 5:
                scores.append(3)
            elif diff < 10:
                scores.append(1)

            elif diff < 20:
                scores.append(0)
            else:
                scores.append(-3)
        except Exception as e:
            print(e)
            scores.append(-3)
    print()
    print("Prompt:")
    print(question)
    print()
    print("Label:")
    print(answer[0])
    print()
    print("Generated:")
    print(generated_answers[0])
    return scores

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [34]:
max_prompt_length = 300 # + 1 just in case!

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 2, # Increase to 4 for smoother training
    num_generations = 2, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 1000,
    save_steps = 5,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "/content/drive/MyDrive/llm_distill/outputs/5_12_a",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 2


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        check_answer
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 4 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 17,432,576/7,000,000,000 (0.25% trained)


invalid literal for int() with base 10: 'No Final Answer'
invalid literal for int() with base 10: 'No Final Answer'

Prompt:
我正在模拟一个智能体的一天。这个智能体名字叫张洁茹，女性，37岁。最高学历为本科。婚姻状态为已婚。当前的人生阶段为自由职业。在星期四凌晨0点到上午九点这9个小时中，她睡觉1.5个小时。在早上6点到9点这3个小时中，她吃饭0.5个小时。在上午11点到下午1点这两个小时中，她吃饭2.0小时。在早上9点到下午6点这9小时中，她娱乐2.0个小时，工作4.0小时，学习3.0小时，。请思考每个提到的动作持续时间的合理性并给出一个分数。0分为非常不正常，100分为非常正常。最后将这些单项的分数求均值得到一个综合起来的分数。在最终的回复中只给出这个综合的分数，不给出任何的理由。

Label:
67

Generated:
No Final Answer
invalid literal for int() with base 10: 'No Final Answer'
invalid literal for int() with base 10: '<tool_call>\n\n65.5'

Prompt:
我正在模拟一个智能体的一天。这个智能体名字叫沈婷洋，女性，32岁。最高学历为大专。婚姻状态为已婚。当前的人生阶段为上班族。在星期三凌晨0点到上午九点这9个小时中，她睡觉9.0个小时。在早上6点到9点这3个小时中，她吃饭3.0个小时。在早上9点到中午12点这3小时中，她娱乐0.75个小时，工作0.5小时，学习0.25小时，。请思考每个提到的动作持续时间的合理性并给出一个分数。0分为非常不正常，100分为非常正常。最后将这些单项的分数求均值得到一个综合起来的分数。在最终的回复中只给出这个综合的分数，不给出任何的理由。

Label:
59

Generated:
No Final Answer


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / match_format_exactly,rewards / check_answer
1,-0.000000,-0.750000,1.060660,474.500000,0.000000,2.250000,-3.000000
2,0.000000,0.000000,0.000000,277.750000,0.000000,3.000000,-3.000000
3,0.000000,-0.750000,1.060660,486.000000,0.000080,2.250000,-3.000000
4,0.000000,-2.250000,1.060660,703.500000,0.000023,0.750000,-3.000000
5,0.000000,-1.500000,2.121320,467.250000,0.000018,1.500000,-3.000000
6,0.000000,-1.500000,2.121320,534.000000,0.000021,1.500000,-3.000000
7,0.000000,-0.750000,3.181980,484.500000,0.000549,1.500000,-2.250000
8,0.000000,-2.250000,1.060660,552.250000,0.000028,0.750000,-3.000000
9,0.000000,-1.500000,2.121320,664.250000,0.000020,1.500000,-3.000000
10,0.000000,-0.750000,1.060660,454.500000,0.000142,2.250000,-3.000000


invalid literal for int() with base 10: 'No Final Answer'
invalid literal for int() with base 10: '<tool_call>\n\n58'

Prompt:
我正在模拟一个智能体的一天。这个智能体名字叫邓琳莎，女性，4岁。最高学历为小学及以下。婚姻状态为未婚。当前的人生阶段为学龄前。在星期三凌晨0点到上午九点这9个小时中，她睡觉4.75个小时。在早上6点到9点这3个小时中，她吃饭0.5个小时。在上午11点到下午1点这两个小时中，她吃饭0.25小时。在早上9点到下午6点这9小时中，她娱乐2.0个小时，学习3.75小时，。请思考每个提到的动作持续时间的合理性并给出一个分数。0分为非常不正常，100分为非常正常。最后将这些单项的分数求均值得到一个综合起来的分数。在最终的回复中只给出这个综合的分数，不给出任何的理由。

Label:
43

Generated:
No Final Answer
invalid literal for int() with base 10: '75.0'

Prompt:
我正在模拟一个智能体的一天。这个智能体名字叫邓强伟，男性，33岁。最高学历为本科。婚姻状态为已婚。当前的人生阶段为自由职业。在星期二凌晨0点到上午九点这9个小时中，他睡觉7.5个小时。在早上6点到9点这3个小时中，他吃饭2.75个小时。在上午11点到下午1点这两个小时中，他吃饭2.0小时。在下午4点到下午8点这四个小时中，他吃饭2.5小时。在早上9点到下午6点这9小时中，他娱乐0.0个小时，工作2.0小时，学习0.0小时，在下午7点到晚上12点这5个小时中，他烹饪8.25小时,他在线学习0.75小时,他吃饭0.0小时,他居家办公0.0小时,他休闲娱乐0.0小时,他洗澡0.0小时,他居家健身0.0小时,他吹头发0.0小时,他睡觉0.0小时,他做家务0.0小时,。请思考每个提到的动作持续时间的合理性并给出一个分数。0分为非常不正常，100分为非常正常。最后将这些单项的分数求均值得到一个综合起来的分数。在最终的回复中只给出这个综合的分数，不给出任何的理由。

Label:
34

Generated:
75.0
invalid literal for int() with base 1

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role": "user", "content": "What is the sqrt of 101?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.79it/s, est. speed input: 211.51 toks/s, output: 67.29 toks/s]


'The square root of 101 is approximately 10.05.'

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Verify LoRA is actually trained!

In [ ]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

Now we load the LoRA and test:

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.21s/it, est. speed input: 39.47 toks/s, output: 77.58 toks/s]


"<start_working_out>\nTo find the square root of 101, we can use a calculator or estimate it manually. Since 101 is not a perfect square, we'll have to use a calculator for an accurate result.\n\nUsing a calculator, the square root of 101 is approximately 10.05.\n\nAlternatively, we can estimate it manually by looking for the closest perfect squares. The closest perfect squares are 100 and 121. Since 101 falls between them, its square root will be between 10 and 11.\n\n10^2 = 100\n11^2 = 121\n\n101 is closer to 121, so the square root of 101 will be slightly less than 11.\n\n10.05 is the estimated square root of 101.\n\n<end_working_out>\n<SOLUTION>10.05</SOLUTION>"

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
